In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv("weather_features.csv")
df = df[['dt_iso', 'city_name', 'wind_speed', 'wind_deg', 'clouds_all', 'weather_main', 'weather_description', 'weather_id']].copy()
df

,dt_iso,city_name,wind_speed,wind_deg,clouds_all,weather_main,weather_description,weather_id
0,2015-01-01 00:00:00+01:00,Valencia,1,62,0,clear,sky is clear,800
1,2015-01-01 01:00:00+01:00,Valencia,1,62,0,clear,sky is clear,800
2,2015-01-01 02:00:00+01:00,Valencia,0,23,0,clear,sky is clear,800
3,2015-01-01 03:00:00+01:00,Valencia,0,23,0,clear,sky is clear,800
4,2015-01-01 04:00:00+01:00,Valencia,0,23,0,clear,sky is clear,800
...,...,...,...,...,...,...,...,...
178391,2018-12-31 19:00:00+01:00,Seville,3,30,0,clear,sky is clear,800
178392,2018-12-31 20:00:00+01:00,Seville,3,30,0,clear,sky is clear,800
178393,2018-12-31 21:00:00+01:00,Seville,4,50,0,clear,sky is clear,800
178394,2018-12-31 22:00:00+01:00,Seville,4,60,0,clear,sky is clear,800


In [4]:
unique_descriptions = df['weather_description'].unique()
unique_ids = df['weather_id'].unique()

print(f"Ukupno ima {len(unique_ids)} različitih idjeva vremena:\n")
print(f"Ukupno ima {len(unique_descriptions)} različitih opisa vremena:\n")
for desc in unique_descriptions:
    print(f"- {desc}")

Ukupno ima 38 različitih idjeva vremena:

Ukupno ima 43 različitih opisa vremena:

- sky is clear
- few clouds
- scattered clouds
- broken clouds
- overcast clouds
- light rain
- moderate rain
- heavy intensity rain
- mist
- heavy intensity shower rain
- shower rain
- very heavy rain
- thunderstorm with heavy rain
- thunderstorm with light rain
- thunderstorm with rain
- proximity thunderstorm
- thunderstorm
- light intensity shower rain
- light intensity drizzle
- fog
- drizzle
- smoke
- heavy intensity drizzle
- haze
- proximity shower rain
- light intensity drizzle rain
- light snow
- rain and snow
- light rain and snow
- snow
- light thunderstorm
- heavy snow
- sleet
- rain and drizzle
- shower sleet
- light shower sleet
- light shower snow
- proximity moderate rain
- ragged shower rain
- sand dust whirls
- proximity drizzle
- dust
- squalls


In [5]:
print(df.columns[df.isnull().any()])

Index([], dtype='object')


In [6]:
df_energy = pd.read_csv("energy_dataset.csv")
df_energy = df_energy[['time', 'generation solar', 'generation wind onshore', 'total load actual']].copy()
df_energy

,time,generation solar,generation wind onshore,total load actual
0,2015-01-01 00:00:00+01:00,49.0,6378.0,25385.0
1,2015-01-01 01:00:00+01:00,50.0,5890.0,24382.0
2,2015-01-01 02:00:00+01:00,50.0,5461.0,22734.0
3,2015-01-01 03:00:00+01:00,50.0,5238.0,21286.0
4,2015-01-01 04:00:00+01:00,42.0,4935.0,20264.0
...,...,...,...,...
35059,2018-12-31 19:00:00+01:00,85.0,3113.0,30653.0
35060,2018-12-31 20:00:00+01:00,33.0,3288.0,29735.0
35061,2018-12-31 21:00:00+01:00,31.0,3503.0,28071.0
35062,2018-12-31 22:00:00+01:00,31.0,3586.0,25801.0


In [7]:
print(df_energy.columns[df_energy.isnull().any()])

Index(['generation solar', 'generation wind onshore', 'total load actual'], dtype='object')


In [8]:
total_rows = len(df_energy)
missing_percentage = (df_energy.isnull().sum() / total_rows) * 100

print("\nProcenat nedostajućih vrednosti:")
print(missing_percentage[missing_percentage > 0].round(2).astype(str) + '%')

df_energy['generation solar'] = df_energy['generation solar'].ffill()
df_energy['generation wind onshore'] = df_energy['generation wind onshore'].ffill()
df_energy['total load actual'] = df_energy['total load actual'].ffill()




Procenat nedostajućih vrednosti:
generation solar           0.05%
generation wind onshore    0.05%
total load actual           0.1%
dtype: object


Moramo osigurati da oba DataFrame-a imaju isto ime kolone za vreme

In [9]:
df_energy = df_energy.rename(columns={'time': 'dt_iso'})

df_energy['dt_iso'] = pd.to_datetime(df_energy['dt_iso'], utc=True)
df['dt_iso'] = pd.to_datetime(df['dt_iso'], utc=True)

Da bismo imali jedan red po satu, moramo raširiti gradove u kolone. Tako ćemo u jednom redu imati temperaturu i vetar za Madrid, Bilbao, Sevilju, Barselonu i Valensiju. Za neke gradove su postojali duplikati vremenskih zapisa za isti sat, pa je bilo potrebno izbrisati duplikate.

In [10]:
# 1. Osiguraj da su SVE potrebne kolone tu (uključujući city_name i weather_id)
# Dodajemo weather_id u listu!
df = df[['dt_iso', 'city_name', 'wind_speed', 'wind_deg', 'clouds_all', 'weather_id']].copy()

# 2. Brisanje duplikata (da pivot ne bi pukao)
df = df.drop_duplicates(subset=['dt_iso', 'city_name'])

df['city_name'] = df['city_name'].str.strip()
# 3. Pivotiranje
df_weather_wide = df.pivot(index='dt_iso', columns='city_name')

# 4. Sređivanje naziva kolona (ovo će sada napraviti weather_id_Barcelona, weather_id_Madrid...)
df_weather_wide.columns = [f'{col[0]}_{col[1]}' for col in df_weather_wide.columns]
df_weather_wide = df_weather_wide.reset_index()

# 5. Ponovni merge sa energetskim podacima
df_final = pd.merge(df_weather_wide, df_energy, on='dt_iso', how='inner')

In [11]:
df_final = pd.merge(df_weather_wide, df_energy, on='dt_iso', how='inner')

# Provera rezultata
print(f"Dimenzije spojenog fajla: {df_final.shape}")

print(df_final.columns.tolist())

Dimenzije spojenog fajla: (35064, 24)
['dt_iso', 'wind_speed_Barcelona', 'wind_speed_Bilbao', 'wind_speed_Madrid', 'wind_speed_Seville', 'wind_speed_Valencia', 'wind_deg_Barcelona', 'wind_deg_Bilbao', 'wind_deg_Madrid', 'wind_deg_Seville', 'wind_deg_Valencia', 'clouds_all_Barcelona', 'clouds_all_Bilbao', 'clouds_all_Madrid', 'clouds_all_Seville', 'clouds_all_Valencia', 'weather_id_Barcelona', 'weather_id_Bilbao', 'weather_id_Madrid', 'weather_id_Seville', 'weather_id_Valencia', 'generation solar', 'generation wind onshore', 'total load actual']


S obzirom da imamo uzorke za vreme po gradovima, a proizvedenu vetro i solarnu energiju na nivou države, moramo uzeti u obzir lokaciju gradova i većih centara za proizvodnju ovih vidova energije. Shodno tome dodelićemo težine svakom od gradova:

In [16]:
weights_wind = {
    'Bilbao': 0.35,   # Predstavlja severozapad (Galicia/Baskija)
    'Madrid': 0.30,   # Predstavlja unutrašnjost (Castilla y León/La Mancha)
    'Valencia': 0.15, # Predstavlja istok i Aragon
    'Seville': 0.15,  # Jug ima vetra, ali manje instalacija
    'Barcelona': 0.05 
}

# SOLAR: Fokus na jug (Sevilla) i suvu unutrašnjost (Madrid/Valencia)
# Sevilla i Valencia dobijaju veći udeo jer imaju najviše sunčanih sati.
weights_solar = {
    'Seville': 0.40,   # Andaluzija
    'Madrid': 0.35,    # Proxy za Castilla-La Mancha i Extremaduru
    'Valencia': 0.15,  # Mediteran
    'Barcelona': 0.05,
    'Bilbao': 0.05
}

df_final['national_wind_index'] = 0
df_final['national_solar_index'] = 0

cities = ['Madrid', 'Bilbao', 'Seville', 'Barcelona', 'Valencia']

for city in cities:

    v = df_final[f'wind_speed_{city}']
    
    # 1. Cut-in (obično 2.5 m/s) i Cut-out (obično 25 m/s)
    # Sve ispod 2 i iznad 25 postaje 0
    v_eff = np.where((v > 2) & (v < 25), v, 0)
    
    # 2. Kubiranje efektivne brzine
    v3 = v_eff**3
    
    # 3. Primena novih težina
    df_final['national_wind_index'] += v3 * weights_wind[city]
    
    is_clear = (df_final[f'weather_id_{city}'] == 800).astype(int)
    solar_potential = (100 - df_final[f'clouds_all_{city}']) * (1 + 0.2 * is_clear)
    df_final['national_solar_index'] += solar_potential * weights_solar[city]

print("Indeksi su uspešno izračunati!")

Indeksi su uspešno izračunati!


In [18]:
df_final['hour'] = df_final['dt_iso'].dt.hour

# Uzimamo samo dnevne sate
df_daylight = df_final[(df_final['hour'] >= 10) & (df_final['hour'] <= 16)]

# Ponovna korelacija samo za dan
corr_solar_day = df_daylight['national_solar_index'].corr(df_daylight['generation solar'])
print(f"Dnevna korelacija za SOLAR: {corr_solar_day:.4f}")

corr_wind = df_final['national_wind_index'].corr(df_final['generation wind onshore'])

print(f"Korelacija za VETAR: {corr_wind:.4f}")
df_final


Dnevna korelacija za SOLAR: 0.3445
Korelacija za VETAR: 0.2840


,dt_iso,wind_speed_Barcelona,wind_speed_Bilbao,wind_speed_Madrid,wind_speed_Seville,wind_speed_Valencia,wind_deg_Barcelona,wind_deg_Bilbao,wind_deg_Madrid,wind_deg_Seville,...,weather_id_Bilbao,weather_id_Madrid,weather_id_Seville,weather_id_Valencia,generation solar,generation wind onshore,total load actual,national_wind_index,national_solar_index,hour
0,2014-12-31 23:00:00+00:00,7,0,1,1,1,58,226,309,21,...,800,800,800,800,49.0,6378.0,25385.0,17.15,120.0,23
1,2015-01-01 00:00:00+00:00,7,0,1,1,1,58,229,309,21,...,800,800,800,800,50.0,5890.0,24382.0,17.15,120.0,0
2,2015-01-01 01:00:00+00:00,7,1,1,3,0,48,224,273,27,...,800,800,800,800,50.0,5461.0,22734.0,21.20,120.0,1
3,2015-01-01 02:00:00+00:00,7,1,1,3,0,48,225,273,27,...,800,800,800,800,50.0,5238.0,21286.0,21.20,120.0,2
4,2015-01-01 03:00:00+00:00,7,1,1,3,0,48,221,273,27,...,800,800,800,800,42.0,4935.0,20264.0,21.20,120.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,2018-12-31 18:00:00+00:00,1,0,1,3,2,250,57,280,30,...,800,800,800,800,85.0,3113.0,30653.0,4.05,120.0,18
35060,2018-12-31 19:00:00+00:00,3,1,1,3,1,270,0,260,30,...,800,800,800,800,33.0,3288.0,29735.0,5.40,120.0,19
35061,2018-12-31 20:00:00+00:00,4,1,1,4,3,300,140,340,50,...,800,800,800,800,31.0,3503.0,28071.0,16.85,120.0,20
35062,2018-12-31 21:00:00+00:00,5,1,2,4,2,320,120,340,60,...,800,800,800,800,31.0,3586.0,25801.0,15.85,120.0,21
